In [ ]:
pip install --upgrade wandb # wandb설치 

In [ ]:
pip install torchinfo # torchinfo 설치

# 문제 1
> ## Fashion	MNIST	데이터 정규화를 위한 Mean과	Std.	값 찾기

In [ ]:
import os
from pathlib import Path
import torch
import wandb
from torch import nn

In [ ]:
wandb.login()

In [ ]:
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets
from torchvision.transforms import transforms

In [ ]:
#BASE_PATH = str(Path(__file__).resolve().parent.parent.parent)  # BASE_PATH: /Users/yhhan/git/link_dl
BASE_PATH = str(Path.cwd().resolve().parent.parent) # Path.cwd() : jupyter notebook에서 Path(__file__)의 기능과 동일하게 제공함
print(BASE_PATH)

In [ ]:
import sys

In [ ]:
sys.path.append(BASE_PATH)

In [ ]:
from _01_code._99_common_utils.utils import get_num_cpu_cores, is_linux, is_windows

In [ ]:
def get_fashion_mnist_data():
    data_path = os.path.join(BASE_PATH, "_00_data", "j_fashion_mnist")
    
    # FashionMNIST 데이터셋을 불러와 data_path에 존재하지 않다면 다운로드. 학습용이므로 suffle, transform 사용.
    f_mnist_train = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.ToTensor())
    f_mnist_train, f_mnist_validation = random_split(f_mnist_train, [55_000, 5_000])
    

    print("Num Train Samples: ", len(f_mnist_train))
    print("Num Validation Samples: ", len(f_mnist_validation))
    print("Sample Shape: ", f_mnist_train[0][0].shape)  # torch.Size([1, 28, 28])

    num_data_loading_workers = get_num_cpu_cores() if is_linux() or is_windows() else 0
    print("Number of Data Loading Workers:", num_data_loading_workers)
    
    img_t, _ = f_mnist_train[0]
    print(f"type : {type(img_t)}")
    print(f"shape : {img_t.shape}")
    print(f"min : {img_t.min()} max : {img_t.max()}")
    
    imgs = torch.stack([img_t for img_t, _ in f_mnist_train], dim=3)
    
    print(f"mean : {imgs.view(1, -1).mean(dim=-1)}")

    print(f"std : {imgs.view(1, -1).std(dim=-1)}")
    

    train_data_loader = DataLoader( 
        dataset=f_mnist_train, batch_size=wandb.config.batch_size, shuffle=True,
        pin_memory=True, num_workers=num_data_loading_workers
    )

    validation_data_loader = DataLoader(
        dataset=f_mnist_validation, batch_size=wandb.config.batch_size,
        pin_memory=True, num_workers=num_data_loading_workers
    )

    f_mnist_transforms = nn.Sequential( 
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=0.2859, std=0.3529), # 데이터 정규화 및 torch 타입으로 변경함 ( 필수 요소 )
    )

    return train_data_loader, validation_data_loader, f_mnist_transforms

In [ ]:
def get_fashion_mnist_test_data():
    data_path = os.path.join(BASE_PATH, "_00_data", "j_fashion_mnist")
    
    # 이미지 데이터 그대로 사용하기 때문에 transform을 사용하지 않음
    f_mnist_test_images = datasets.FashionMNIST(data_path, train=False, download=True)
    f_mnist_test = datasets.FashionMNIST(data_path, train=False, download=True, transform=transforms.ToTensor())

    print("Num Test Samples: ", len(f_mnist_test))
    print("Sample Shape: ", f_mnist_test[0][0].shape)  # torch.Size([1, 28, 28])

    test_data_loader = DataLoader(dataset=f_mnist_test, batch_size=len(f_mnist_test))

    f_mnist_transforms = nn.Sequential(
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=0.2859, std=0.3529), # torch 형태로 바꾸고, 정규화 과정을 거침 (필수요소)
    )

    return f_mnist_test_images, test_data_loader, f_mnist_transforms

In [ ]:
from _01_code._08_fcn_best_practice.c_trainer import ClassificationTrainer
from _01_code._08_fcn_best_practice.f_mnist_train_fcn import get_mnist_data
from _01_code._08_fcn_best_practice.e_arg_parser import get_parser

In [ ]:
if __name__ == "__main__":
    config = {'batch_size': 2048, }
    wandb.init(mode="online", config=config)
    
    # train, validation dataset을 불러옴
    train_data_loader, validation_data_loader, f_mnist_transforms = get_fashion_mnist_data() 
    print()
    f_mnist_test_images, test_data_loader, f_mnist_transforms = get_fashion_mnist_test_data() # test dataset을 불러옴

# 문제 2
> # Fashion	MNIST	데이터에	대하여	CNN	학습시키기

In [ ]:
from datetime import datetime
import torch
from torch import nn

from _01_code._99_common_utils.early_stopping import EarlyStopping
from _01_code._99_common_utils.utils import strfdelta
from torch.optim import lr_scheduler

class ClassificationTrainer:
  def __init__(
    self, project_name, model, optimizer, train_data_loader, validation_data_loader, transforms,
    run_time_str, wandb, device, checkpoint_file_path
  ):
    self.project_name = project_name
    self.model = model
    self.optimizer = optimizer
    self.train_data_loader = train_data_loader
    self.validation_data_loader = validation_data_loader
    self.transforms = transforms
    self.run_time_str = run_time_str
    self.wandb = wandb
    self.device = device
    self.checkpoint_file_path = checkpoint_file_path

    # Use a built-in loss function
    self.loss_fn = nn.CrossEntropyLoss() # 다중 분류에 사용되는 loss 함수
    
    self.scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1) # 100 epoch 마다 lr을 0.1정도 줄이는 schedular 정의

  def do_train(self):
    self.model.train()  # Will be explained at 'Diverse Techniques' section / 모델 학습에선 꼭 사용해야함.

    loss_train = 0.0
    num_corrects_train = 0
    num_trained_samples = 0
    num_trains = 0

    for train_batch in self.train_data_loader:
      input_train, target_train = train_batch
      input_train = input_train.to(device=self.device)
      target_train = target_train.to(device=self.device)

      if self.transforms:
        input_train = self.transforms(input_train)

      output_train = self.model(input_train)
      loss = self.loss_fn(output_train, target_train)
      loss_train += loss.item()

      predicted_train = torch.argmax(output_train, dim=-1) # 다중 분류에 사용됨.

      # >>> predicted_train: tensor([5, 8, 9, 0, 9, 8, 9, 8, ..., 0, 1, 3, 7, 1, 4, 3])
      # >>> target_train:    tensor([5, 8, 9, 2, 9, 8, 7, 8, ..., 4, 1, 9, 6, 1, 4, 3])
      num_corrects_train += torch.sum(torch.eq(predicted_train, target_train)).item()

      num_trained_samples += len(input_train)
      num_trains += 1

      self.optimizer.zero_grad() # 
      loss.backward() # 
      self.optimizer.step() # 가중치 수정
    
    #self.scheduler.step() # 하나의 epoch가 끝나면 호출

    train_loss = loss_train / num_trains
    train_accuracy = 100.0 * num_corrects_train / num_trained_samples

    return train_loss, train_accuracy

  def do_validation(self):
    self.model.eval()   # Explained at 'Diverse Techniques' section

    loss_validation = 0.0
    num_corrects_validation = 0
    num_validated_samples = 0
    num_validations = 0

    with torch.no_grad():
      for validation_batch in self.validation_data_loader:
        input_validation, target_validation = validation_batch
        input_validation = input_validation.to(device=self.device)
        target_validation = target_validation.to(device=self.device)

        if self.transforms:
          input_validation = self.transforms(input_validation)

        output_validation = self.model(input_validation)
        loss_validation += self.loss_fn(output_validation, target_validation).item()

        predicted_validation = torch.argmax(output_validation, dim=1)
        num_corrects_validation += torch.sum(torch.eq(predicted_validation, target_validation)).item()

        num_validated_samples += len(input_validation)
        num_validations += 1

    validation_loss = loss_validation / num_validations
    validation_accuracy = 100.0 * num_corrects_validation / num_validated_samples

    return validation_loss, validation_accuracy

  def train_loop(self):
    early_stopping = EarlyStopping(
      patience=self.wandb.config.early_stop_patience,
      delta=self.wandb.config.early_stop_delta,
      project_name=self.project_name,
      checkpoint_file_path=self.checkpoint_file_path,
      run_time_str=self.run_time_str
    )
    n_epochs = self.wandb.config.epochs
    training_start_time = datetime.now()

    for epoch in range(1, n_epochs + 1):
      train_loss, train_accuracy = self.do_train()

      if epoch == 1 or epoch % self.wandb.config.validation_intervals == 0:
        validation_loss, validation_accuracy = self.do_validation()

        elapsed_time = datetime.now() - training_start_time
        epoch_per_second = 0 if elapsed_time.seconds == 0 else epoch / elapsed_time.seconds

        message, early_stop = early_stopping.check_and_save(validation_loss, self.model)

        print(
          f"[Epoch {epoch:>3}] "
          f"T_loss: {train_loss:7.5f}, "
          f"T_accuracy: {train_accuracy:6.4f} | "
          f"V_loss: {validation_loss:7.5f}, "
          f"V_accuracy: {validation_accuracy:6.4f} | "
          f"{message} | "
          f"T_time: {strfdelta(elapsed_time, '%H:%M:%S')}, "
          f"T_speed: {epoch_per_second:4.3f}"
        )

        self.wandb.log({
          "Epoch": epoch,
          "Training loss": train_loss,
          "Training accuracy (%)": train_accuracy,
          "Validation loss": validation_loss,
          "Validation accuracy (%)": validation_accuracy,
          "Training speed (epochs/sec.)": epoch_per_second,
        })

        if early_stop:
          break

    elapsed_time = datetime.now() - training_start_time
    print(f"Final training time: {strfdelta(elapsed_time, '%H:%M:%S')}")


In [ ]:
import os
from pathlib import Path
import torch
import wandb
from torch import nn, optim
from datetime import datetime

from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets
from torchvision.transforms import transforms
from torchinfo import summary

# BASE_PATH: /Users/yhhan/git/link_dl
BASE_PATH = str(Path.cwd().resolve().parent.parent)
print(BASE_PATH)

CURRENT_FILE_PATH = os.path.abspath(os.getcwd())
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH):
    os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

import sys

sys.path.append(BASE_PATH)

from _01_code._99_common_utils.utils import get_num_cpu_cores, is_linux, is_windows
from _01_code._08_fcn_best_practice.e_arg_parser import get_parser



def get_fashion_mnist_data():
    data_path = os.path.join(BASE_PATH, "_00_data", "j_fashion_mnist")

    # FashionMNIST 데이터셋을 불러와 data_path에 존재하지 않다면 다운로드 학습용으로 Tensor화 시켜서 저장
    f_mnist_train = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.RandomRotation(10) # 기존 가져온 데이터셋에 data argumentation을 적용
                                      ]))                               # 회전을 적용한 후 텐서화 해 저장함.
    f_mnist_train, f_mnist_validation = random_split(f_mnist_train, [55_000, 5_000])
    
    
    

    print("Num Train Samples: ", len(f_mnist_train))
    print("Num Validation Samples: ", len(f_mnist_validation))
    print("Sample Shape: ", f_mnist_train[0][0].shape)  # torch.Size([1, 28, 28])

    num_data_loading_workers = get_num_cpu_cores() if is_linux() or is_windows() else 0
    print("Number of Data Loading Workers:", num_data_loading_workers)

    img_t, _ = f_mnist_train[0]
    print(f"type : {type(img_t)}")
    print(f"shape : {img_t.shape}")
    print(f"min : {img_t.min()} max : {img_t.max()}")

    imgs = torch.stack([img_t for img_t, _ in f_mnist_train], dim=3)

    print(f"mean : {imgs.view(1, -1).mean(dim=-1)}")

    print(f"std : {imgs.view(1, -1).std(dim=-1)}")

    train_data_loader = DataLoader(
        dataset=f_mnist_train, batch_size=wandb.config.batch_size, shuffle=True,
        pin_memory=True, num_workers=num_data_loading_workers
    )

    validation_data_loader = DataLoader(
        dataset=f_mnist_validation, batch_size=wandb.config.batch_size,
        pin_memory=True, num_workers=num_data_loading_workers
    )

    f_mnist_transforms = nn.Sequential(
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=0.2859, std=0.3529), # mean, std값이 계속 -2~-2 정도의 변화가 있음
        
    )

    return train_data_loader, validation_data_loader, f_mnist_transforms

def get_fashion_mnist_test_data():
    data_path = os.path.join(os.path.pardir, os.path.pardir, "_00_data", "j_fashion_mnist")

    f_mnist_test_images = datasets.FashionMNIST(data_path, train=False, download=True)
    f_mnist_test = datasets.FashionMNIST(data_path, train=False, download=True, transform=transforms.ToTensor())

    print("Num Test Samples: ", len(f_mnist_test))
    print("Sample Shape: ", f_mnist_test[0][0].shape)  # torch.Size([1, 28, 28])

    test_data_loader = DataLoader(dataset=f_mnist_test, batch_size=len(f_mnist_test))

    f_mnist_transforms = nn.Sequential(
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=0.2859, std=0.3529),
    )

    return f_mnist_test_images, test_data_loader, f_mnist_transforms

def get_cnn_model():
    
    class MyModel(nn.Module):
            def __init__(self, in_channels, n_output):
                
                super().__init__()

                self.model = nn.Sequential(
                # B x 1 x 28 x 28 --> B x 6 x (28 - 5 + 1) x (28 - 5 + 1) = B x 6 x 24 x 24
                nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=(5, 5), stride=(1, 1)),
                # B x 6 x 24 x 24 --> B x 6 x 12 x 12
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.BatchNorm2d(num_features=6, eps=1e-05, momentum=0.1), # 데이터 정규화 과정 진행
                nn.ReLU(),
                # B x 6 x 12 x 12 --> B x 16 x (12 - 5 + 1) x (12 - 5 + 1) = B x 16 x 8 x 8
                nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1)),
                # B x 16 x 8 x 8 --> B x 16 x 4 x 4
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.ReLU(),        
                nn.Flatten(),
                nn.Dropout(p=0.1), # 일정 확률로 뉴런을 배제함.
                nn.Linear(256, 128),
                nn.BatchNorm1d(num_features=128, eps=1e-05, momentum=0.1), # 데이터 정규화 과정 진행
                nn.ReLU(),
                nn.Dropout(p=0.1), # 일정 확률로 뉴런을 배제함.
                nn.Linear(128, n_output),
              )

            def forward(self, x):
              x = self.model(x)
              return x

      # 1 * 28 * 28
    my_model = MyModel(in_channels=1, n_output=10)

    return my_model




def main(args):
    
    run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'validation_intervals': args.validation_intervals, # validation을 몇번 진행할 때마다 early_stop을 검사하는지에 대한 값
        'learning_rate': 1e-3, #args.learning_rate, # 학습률
        'early_stop_patience': 20, #args.early_stop_patience, # early_stop이 발생하려고 할 때 몇 번 참을 건지에 대한 값.
        'early_stop_delta': 0.001 #args.early_stop_delta # 이전 값보다 어느정도 차이가 벌어졌을 때 early_stop을 진행할 건지.
      }

    project_name = "cnn_f_mnist"
    wandb.init(
        mode="online",# if args.wandb else "disabled",
        project=project_name,
        notes="f_mnist Homework with cnn",
        tags=["cnn", "f_mnist"],
        name=run_time_str,
        config=config
      )
    print(args)
    print(wandb.config)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Training on device {device}.")

    train_data_loader, validation_data_loader, f_mnist_transforms = get_fashion_mnist_data()
    print()
    f_mnist_test_images, test_data_loader, f_mnist_transforms = get_fashion_mnist_test_data()

    model = get_cnn_model()
    model.to(device)
    wandb.watch(model)


    info = summary(model=model, input_size=(1, 1, 28, 28)) # 모델에 대항 정보를 출력함. input_size는 정확하게 입력해야 오류가 없음
    print(info) # jupyter notebook은 summary만 설정해선 출력되지 않아 임의로 출력시킴.
    optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate, weight_decay=0.0005) # Adam optimizer 사용.

    classification_trainer = ClassificationTrainer(
        project_name, model, optimizer, train_data_loader, validation_data_loader, f_mnist_transforms,
        run_time_str, wandb, device, CHECKPOINT_FILE_PATH
      )
    classification_trainer.train_loop()

    wandb.finish()

    

if __name__ == "__main__":
    parser = get_parser()
    args = parser.parse_args([])
    main(args)

# 모델 학습에 대한 고찰

- 기본 조건으로 학습 시
> validation accuracy 88%
- adam, batch normalization, drop, data augmentation, regularzation을 무작위 값으로 적용한 후
> validation accuracy 89.46% | test accuracy 89.070% <br>
> EARLY STOPPED : 570
- ReLU -> LeakyReLu, lr decay 추가로 사용한 후
> validation accuracy 89.02% | test accuracy 89.210%
> LeakyReLu를 사용할 때가 오히려 ReLU보다 나빠지고 있다는 것을 알 수 있음
> fashion mnist 데이터에게만 ReLU가 더 유리한건지, 아니면 이미지 데이터 전체에게 ReLU가 더 유리한건지 생각할 수 있음.
- data augmentation 1개 추가 적용 
> validation acuuracy 88.09% | test accuracy 89.0%
> data augmentation을 여러개 적용하니 오히려 떨어지는 것을 알 수 있었음.
> 데이터가 더 복잡해졌기 때문에 과소적합이 아닌가 싶기도 하고 lr을 늘리면 더 좋아지지 않을까 생각됨.
- drop: 0.5 -> 0.4, learning rate 1e-3 -> 1e-4, batch norm-momentum 0.1 -> 0.5
> validation acuuracy 87.7% | test accuracy 87.23%
> 이전보다 많이 낮아졌기 때문에 lr을 낮추고 batch normalization의 momentum을 높이는 것은 지양.
- batch norm_momentum 0.5 -> 0.1, eps 1e-5 -> 1e-3
> validation acuuracy 87.46% | test accuracy 87.440%
> batch normalization값을 변경하는 것은 성능을 끌어올리는데 큰 영향을 주지 않는 것으로 생각됨.
- batch norm_mometum 0.1 -> 0.9
> validation acuuracy 86.14% | test accuracy 87.350%
> 역시 normalization momentum값은 기존 값으로 유지
- dropout => 0.1, weight_decay => 0.0005
> 정확도가 어느정도 올라감.
> dropout값과 weight_decay값은 낮추는게 모델 성능에 도움이 되는 것으로 생각함.
- LeakyReLU -> ReLU
> 예상대로 성능이 올라감
- batch size : 2048 -> 1024
> 모델 성능이 기존 91.7%에서 90.7%로 떨어짐
- batch size : 2048 -> 3072
> 역시 모델 성능이 기존보다 떨어짐
> batch size는 기존 2048이 맞는듯
- schedular 추가 ( 10 epoch 마다 실행)
> 학습이 기존보다 떨어짐
- schedluar : 10 epoch -> 100 epoch
> 학습이 기존보다 떨어짐
> schedular를 사용하지 않던지, stepLR말고 다른 schedular를 사용해야 할것으로 생각됨.
- transform에서 nn.Sequential 대신 transforms.Compose를 사용할 수도 있다는 것을 알 수 있었음. 

# 문제 3, 4
> # 학습 완료된 모델로 테스트 데이터 Accuracy	확인하기
> # 샘플 테스트 데이터 분류 예측 결과 확인하기

In [ ]:
import os
from pathlib import Path
import torch
import wandb
from torch import nn, optim
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np

from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import transforms

# BASE_PATH: /Users/yhhan/git/link_dl
BASE_PATH = str(Path.cwd().resolve().parent.parent)
print(BASE_PATH)

CURRENT_FILE_PATH = os.path.abspath(os.getcwd())
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH):
    os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

import sys

sys.path.append(BASE_PATH)



from _01_code._08_fcn_best_practice.d_tester import ClassificationTester # 기존 코드 ClassificationTester를 그대로 사용함



def main():
    f_mnist_test_images, test_data_loader, f_mnist_transforms = get_fashion_mnist_test_data()
    test_model = get_cnn_model()
    classification_tester = ClassificationTester(
        "cnn_f_mnist", test_model, test_data_loader, f_mnist_transforms, CHECKPOINT_FILE_PATH
      )
    classification_tester.test()

    print()
    
    for i in range(9):
        img, label = f_mnist_test_images[i]
        print("     LABEL:", label)
        plt.imshow(img)
        plt.show()

        output = classification_tester.test_single(
        torch.tensor(np.array(f_mnist_test_images[i][0])).unsqueeze(dim=0).unsqueeze(dim=0))
        print("PREDICTION:", output)
    
    j = 12
    img, label = f_mnist_test_images[j]
    print("     LABEL:", label)
    plt.imshow(img)
    plt.show()

    torch.tensor(np.array(f_mnist_test_images[j][0])).unsqueeze(dim=0).unsqueeze(dim=0).shape: (1, 1, 28, 28)
    output = classification_tester.test_single(
        torch.tensor(np.array(f_mnist_test_images[j][0])).unsqueeze(dim=0).unsqueeze(dim=0)
      )
    print("PREDICTION:", output)


if __name__ == "__main__":
    main()


# 테스트 결과
- 9개의 테스트 샘플의 경우 정답을 맞췄지만, 마지막 한 테스트 샘플의 경우 정답을 맞추기 못하였다.
- 7번 sneakers와 5번 sandle은 모두 신발의 형태를 띄고 있어 대략적으로 보면 유사하다고 생각할 수 있을 것 같다.
- 아마 pooling이 부족해서 모델이 세부적인 특징을 학습한 것이 아니라 전체적인 특징을 학습한 것 같다.

# 숙제 후기
> 강의 시간에 하셨던 "경험이 중요하다"라는 내용이 확 와닫는 계기가 되었습니다. 처음엔 단지 모든 방법을 다 추가해서 돌려봤더니 오히려 아무것도 없을 때의 정확도보다 오히려 더 떨어지는 결과가 나왔으며, 그 이후로 값을 늘리거나 줄이면서 어떻게 하면 정확도가 올라가는지 계속 모델을 학습시켜봤습니다. 또한 하나의 요소만 고려하는 것이 아닌, 가령 "dropout 값만 올리면 정확도가 좋아질 수 있다" 이런 것이 아니라 "데이터 종류에 따라 정확도가 높아지는 Layer개수가 다르고 Layer 개수에 따라 좋은 dropout값이 다르다" 처럼 여러 요소들이 서로 엮여있어정확도를 올리기 위한 값을 조정하기 까다로웠습니다. 과제 제출 2주 전부터 모델 학습을 밤새 돌렸는데 학습 시간이 적어도 2시간 많으면 6시간이 걸리다 보니 아직 여러 방법을 시도하지 못해 너무 아쉽다는 생각이 들었고, 모델 정확도를 94%이상으로 올리지 못한 것 역시 너무 아깝다는 생각이 들었습니다. 주피터 노트북에서 모델을 돌려보면서 data argumentation 부분에서 계속 커널이 나가는 경우가 많아 모델 학습이 제한이 걸렸는데, 콘솔로 바꿔서 진행하지 못한 것이 아쉬웠습니다.     